## Language conversion

In [3]:
import pandas as pd
from deep_translator import GoogleTranslator
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
import re

@lru_cache(maxsize=10000)
def translate_text(text, dest='en'):
    """Translate non-English text to English with caching."""
    if is_english_or_numeric(text):
        return text
    try:
        return GoogleTranslator(source='auto', target=dest).translate(text)
    except Exception as e:
        print(f"Translation error: {text} -> {e}")
        return text

def is_english_or_numeric(text):
    return re.match(r'^[a-zA-Z0-9\s.,!?\'"-]*$', text) is not None

def translate_column(column_data):
    return column_data.apply(lambda x: translate_text(str(x)))

def translate_excel_all_sheets(input_file, output_file, sheet_column_map):
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        for sheet_name, columns_to_translate in sheet_column_map.items():
            df = pd.read_excel(input_file, sheet_name=sheet_name)
            translated_df = df.copy()
            with ThreadPoolExecutor(max_workers=len(columns_to_translate)) as executor:
                futures = {
                    executor.submit(translate_column, df[col]): col
                    for col in columns_to_translate if col in df.columns
                }
                for future in futures:
                    col = futures[future]
                    translated_df[f"{col}_translated"] = future.result()
            translated_df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"✅ Translated sheet: {sheet_name}")

# === File paths
input_file = r"C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\1 RAW DATA\combine data set.xlsx"
output_file = r"C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\translated_all_sheets.xlsx"

# === Column configuration per sheet
sheet_column_map = {
    'english': ['Language', 'Canton', 'District', 'Gender', 'Scheme', 'Issue'],
    'Français': ['Langue', 'Canton', 'District', 'Sexe', 'Prestation', 'Problème'],
    'Deutsch 1': ['Sprach', 'Kanton', 'Bezirk', 'Geschlecht', 'Leistung', 'Unterstützung'],
    'Deutsch 2': ['Sprach', 'Kanton', 'Bezirk', 'Geschlecht', 'Leistung', 'Unterstützung']
}

# === Run translation
translate_excel_all_sheets(input_file, output_file, sheet_column_map)


✅ Translated sheet: english
✅ Translated sheet: Français
✅ Translated sheet: Deutsch 1
✅ Translated sheet: Deutsch 2


In [4]:
df = pd.ExcelFile(output_file)

In [5]:
print(df.sheet_names)

['english', 'Français', 'Deutsch 1', 'Deutsch 2']


## English column standardization

In [6]:
df_sheet1 = df.parse('english') 

In [7]:
df_sheet1.head()

,Name,Phone,Language,Canton,District,Gender,Age Group,Scheme,Issue,Timestamp,Language_translated,Canton_translated,District_translated,Gender_translated,Scheme_translated,Issue_translated
0,Julie Gutierrez,+41 69 36 44 78,English,Neuchâtel (NE),Boudry,Male,35–40,Disability Insurance (DI),Other reason,05.12.2024 17:22:20,English,Neuchâtel (ne),Boudry,Male,Disability Insurance (by),Other reason
1,Katelyn Santiago,+41 89 28 15 19,English,Geneva (GE),Geneva,Female,41–46,Health/Accident Insurance (LAMal/LAA),Document/account issue,27.11.2024 06:27:53,English,Geneva (GE),Geneva,Female,Health/Accident Insurance (LAMal/LAA),Document/account issue
2,Edward Moore,+41 89 75 30 08,English,Zurich (ZH),Zurich,Female,35–40,Old-age and Survivors Insurance (OASI),Demand for bribe,27.11.2024 20:42:40,English,Zurich (ZH),Zurich,Female,Old-age and Survivors Insurance (OASI),Demand for bribe
3,Megan Jackson,+41 13 10 29 09,English,Basel-Stadt (BS),Basel-Stadt,Female,53–58,Health/Accident Insurance (LAMal/LAA),Other reason,22.11.2024 16:41:44,English,Basel-Stadt (BS),Basel-Stadt,Female,Health/Accident Insurance (LAMal/LAA),Other reason
4,Jesse Thompson,+41 09 05 53 58,English,Geneva (GE),Geneva,Male,41–46,Unemployment Insurance (UI),Help applying,28.11.2024 02:19:59,English,Geneva (GE),Geneva,Male,Unemployment Insurance (UI),Help applying


In [8]:
df_sheet1 = df_sheet1.drop(columns=['Language_translated','Canton_translated','District_translated','Gender_translated','Scheme_translated','Issue_translated'],errors='ignore')

In [9]:
# Corrected column rename map
column_rename_map = {
    'Phone': 'Mobile Number',
}

# Rename the columns in the DataFrame
df_sheet1.rename(columns=column_rename_map, inplace=True)

In [10]:
df_sheet1.head()

,Name,Mobile Number,Language,Canton,District,Gender,Age Group,Scheme,Issue,Timestamp
0,Julie Gutierrez,+41 69 36 44 78,English,Neuchâtel (NE),Boudry,Male,35–40,Disability Insurance (DI),Other reason,05.12.2024 17:22:20
1,Katelyn Santiago,+41 89 28 15 19,English,Geneva (GE),Geneva,Female,41–46,Health/Accident Insurance (LAMal/LAA),Document/account issue,27.11.2024 06:27:53
2,Edward Moore,+41 89 75 30 08,English,Zurich (ZH),Zurich,Female,35–40,Old-age and Survivors Insurance (OASI),Demand for bribe,27.11.2024 20:42:40
3,Megan Jackson,+41 13 10 29 09,English,Basel-Stadt (BS),Basel-Stadt,Female,53–58,Health/Accident Insurance (LAMal/LAA),Other reason,22.11.2024 16:41:44
4,Jesse Thompson,+41 09 05 53 58,English,Geneva (GE),Geneva,Male,41–46,Unemployment Insurance (UI),Help applying,28.11.2024 02:19:59


In [11]:
new_column_order = [
    "Name", "Mobile Number", "Age Group", "Language", "Gender",
    "Canton", "District", "Scheme", "Issue","Timestamp"
]

# Reorder the columns
df_sheet1 = df_sheet1[new_column_order]

In [12]:
df25 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\english call center segmentation.xlsx')

In [13]:
df25

,Issue,Call center
0,Other reason,Our Call Center
1,Document/account issue,Our Call Center
2,Demand for bribe,MLA/Party Office
3,Help applying,Our Call Center
4,Missing approval SMS,Division Office
5,Disabled assistance,Division Office
6,Pending payment,Our Call Center
7,Form issue,MLA/Party Office
8,Website/Application trouble,MLA/Party Office
9,Benefit issue,Our Call Center


In [14]:
df_sheet1 = pd.merge(df_sheet1, df25, on='Issue', how='left')

In [15]:
df_sheet1.head()

,Name,Mobile Number,Age Group,Language,Gender,Canton,District,Scheme,Issue,Timestamp,Call center
0,Julie Gutierrez,+41 69 36 44 78,35–40,English,Male,Neuchâtel (NE),Boudry,Disability Insurance (DI),Other reason,05.12.2024 17:22:20,Our Call Center
1,Katelyn Santiago,+41 89 28 15 19,41–46,English,Female,Geneva (GE),Geneva,Health/Accident Insurance (LAMal/LAA),Document/account issue,27.11.2024 06:27:53,Our Call Center
2,Edward Moore,+41 89 75 30 08,35–40,English,Female,Zurich (ZH),Zurich,Old-age and Survivors Insurance (OASI),Demand for bribe,27.11.2024 20:42:40,MLA/Party Office
3,Megan Jackson,+41 13 10 29 09,53–58,English,Female,Basel-Stadt (BS),Basel-Stadt,Health/Accident Insurance (LAMal/LAA),Other reason,22.11.2024 16:41:44,Our Call Center
4,Jesse Thompson,+41 09 05 53 58,41–46,English,Male,Geneva (GE),Geneva,Unemployment Insurance (UI),Help applying,28.11.2024 02:19:59,Our Call Center


## French column standardization

In [16]:
df_sheet2 = df.parse('Français')

In [17]:
df_sheet2.head()

,Nom,Téléphone,Langue,Canton,District,Sexe,Tranche d'âge,Prestation,Problème,Horodatage,Langue_translated,Canton_translated,District_translated,Sexe_translated,Prestation_translated,Problème_translated
0,Claude Béguelin,+41 61 17 49 01,Français,Fribourg (FR),Sarine,Homme,41–46,Assurance maladie/accident (LAMal/LAA),Demande de pot-de-vin,05.11.2024 03:52:40,French,Fruit (fr),Sarine,Homme,Health/accident insurance (LAMAL/LAA),Demande de pot-de-vin
1,Odette Chapuis,+41 10 79 23 88,Français,Valais (VS),Viège,Femme,47–52,Assurance invalidité (AI),SMS d'approbation manquant,14.10.2024 17:31:53,French,Valais (vs),Threesome,Femme,Disability insurance (AI),SMS d'approbation manquant
2,Robert Barillon,+41 06 90 09 33,Français,Argovie (AG),Brugg,Homme,29–34,Assurance vieillesse et survivants (AVS),Autre raison,28.11.2024 10:14:14,French,Argovie (AG),Brugg,Homme,Old age and survivors insurance (AVS),Autre raison
3,Philippe Martin,+41 09 53 06 62,Français,Soleure (SO),Thal-Gäu,Homme,23–28,Assurance invalidité (AI),NaN,19.01.2025 06:01:03,French,SOOL (SO),Thal-Gäu,Homme,Disability insurance (AI),NaN
4,Jonathan Mercier,+41 07 81 25 64,Français,Appenzell,Oberegg,Homme,35–40,Assurance chômage (AC),Autre raison,01.01.2025 07:27:26,French,Appenzell,Oberegg,Homme,Unemployment insurance (AC),Autre raison


language

In [19]:
df1 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french language.xlsx')

In [20]:
df_sheet2 = pd.merge(df_sheet2, df1, on='Langue_translated', how='left')

In [21]:
df_sheet2 = df_sheet2.drop(columns=['Langue','Langue_translated'],errors='ignore')

In [22]:
df_sheet2.head(5)

,Nom,Téléphone,Canton,District,Sexe,Tranche d'âge,Prestation,Problème,Horodatage,Canton_translated,District_translated,Sexe_translated,Prestation_translated,Problème_translated,Language
0,Claude Béguelin,+41 61 17 49 01,Fribourg (FR),Sarine,Homme,41–46,Assurance maladie/accident (LAMal/LAA),Demande de pot-de-vin,05.11.2024 03:52:40,Fruit (fr),Sarine,Homme,Health/accident insurance (LAMAL/LAA),Demande de pot-de-vin,French
1,Odette Chapuis,+41 10 79 23 88,Valais (VS),Viège,Femme,47–52,Assurance invalidité (AI),SMS d'approbation manquant,14.10.2024 17:31:53,Valais (vs),Threesome,Femme,Disability insurance (AI),SMS d'approbation manquant,French
2,Robert Barillon,+41 06 90 09 33,Argovie (AG),Brugg,Homme,29–34,Assurance vieillesse et survivants (AVS),Autre raison,28.11.2024 10:14:14,Argovie (AG),Brugg,Homme,Old age and survivors insurance (AVS),Autre raison,French
3,Philippe Martin,+41 09 53 06 62,Soleure (SO),Thal-Gäu,Homme,23–28,Assurance invalidité (AI),NaN,19.01.2025 06:01:03,SOOL (SO),Thal-Gäu,Homme,Disability insurance (AI),NaN,French
4,Jonathan Mercier,+41 07 81 25 64,Appenzell,Oberegg,Homme,35–40,Assurance chômage (AC),Autre raison,01.01.2025 07:27:26,Appenzell,Oberegg,Homme,Unemployment insurance (AC),Autre raison,French


Gender

In [23]:
df2 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french_gender.xlsx')

In [24]:
# Perform the merge
df_sheet2 = pd.merge(df_sheet2, df2, on='Sexe_translated', how='left')

In [25]:
df_sheet2 = df_sheet2.drop(columns=['Sexe','Sexe_translated'],errors='ignore')

In [26]:
df_sheet2.head(5)

,Nom,Téléphone,Canton,District,Tranche d'âge,Prestation,Problème,Horodatage,Canton_translated,District_translated,Prestation_translated,Problème_translated,Language,Gender
0,Claude Béguelin,+41 61 17 49 01,Fribourg (FR),Sarine,41–46,Assurance maladie/accident (LAMal/LAA),Demande de pot-de-vin,05.11.2024 03:52:40,Fruit (fr),Sarine,Health/accident insurance (LAMAL/LAA),Demande de pot-de-vin,French,Male
1,Odette Chapuis,+41 10 79 23 88,Valais (VS),Viège,47–52,Assurance invalidité (AI),SMS d'approbation manquant,14.10.2024 17:31:53,Valais (vs),Threesome,Disability insurance (AI),SMS d'approbation manquant,French,Female
2,Robert Barillon,+41 06 90 09 33,Argovie (AG),Brugg,29–34,Assurance vieillesse et survivants (AVS),Autre raison,28.11.2024 10:14:14,Argovie (AG),Brugg,Old age and survivors insurance (AVS),Autre raison,French,Male
3,Philippe Martin,+41 09 53 06 62,Soleure (SO),Thal-Gäu,23–28,Assurance invalidité (AI),NaN,19.01.2025 06:01:03,SOOL (SO),Thal-Gäu,Disability insurance (AI),NaN,French,Male
4,Jonathan Mercier,+41 07 81 25 64,Appenzell,Oberegg,35–40,Assurance chômage (AC),Autre raison,01.01.2025 07:27:26,Appenzell,Oberegg,Unemployment insurance (AC),Autre raison,French,Male


scheme

In [27]:
df3 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french scheme.xlsx')

In [28]:
df3

,Prestation_translated,Scheme
0,Health/accident insurance (LAMAL/LAA),Health/Accident Insurance (LAMal/LAA)
1,Disability insurance (AI),Disability Insurance (DI)
2,Old age and survivors insurance (AVS),Old-age and Survivors Insurance (OASI)
3,Unemployment insurance (AC),Unemployment Insurance (UI)
4,Professional provident (LPP),Occupational Pension Plan (BVG)
5,Family allowances (AF),Family Allowances (FA)
6,NaN,NaN


In [29]:
# Perform the merge
df_sheet2 = pd.merge(df_sheet2, df3, on='Prestation_translated', how='left')

In [30]:
df_sheet2

,Nom,Téléphone,Canton,District,Tranche d'âge,Prestation,Problème,Horodatage,Canton_translated,District_translated,Prestation_translated,Problème_translated,Language,Gender,Scheme
0,Claude Béguelin,+41 61 17 49 01,Fribourg (FR),Sarine,41–46,Assurance maladie/accident (LAMal/LAA),Demande de pot-de-vin,05.11.2024 03:52:40,Fruit (fr),Sarine,Health/accident insurance (LAMAL/LAA),Demande de pot-de-vin,French,Male,Health/Accident Insurance (LAMal/LAA)
1,Odette Chapuis,+41 10 79 23 88,Valais (VS),Viège,47–52,Assurance invalidité (AI),SMS d'approbation manquant,14.10.2024 17:31:53,Valais (vs),Threesome,Disability insurance (AI),SMS d'approbation manquant,French,Female,Disability Insurance (DI)
2,Robert Barillon,+41 06 90 09 33,Argovie (AG),Brugg,29–34,Assurance vieillesse et survivants (AVS),Autre raison,28.11.2024 10:14:14,Argovie (AG),Brugg,Old age and survivors insurance (AVS),Autre raison,French,Male,Old-age and Survivors Insurance (OASI)
3,Philippe Martin,+41 09 53 06 62,Soleure (SO),Thal-Gäu,23–28,Assurance invalidité (AI),NaN,19.01.2025 06:01:03,SOOL (SO),Thal-Gäu,Disability insurance (AI),NaN,French,Male,Disability Insurance (DI)
4,Jonathan Mercier,+41 07 81 25 64,Appenzell,Oberegg,35–40,Assurance chômage (AC),Autre raison,01.01.2025 07:27:26,Appenzell,Oberegg,Unemployment insurance (AC),Autre raison,French,Male,Unemployment Insurance (UI)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419415,Jacques Niquille,+41 38 91 68 43,Vaud (VD),Nyon,16–22,Assurance invalidité (AI),Autre raison,19.10.2024 08:17:43,Vaud (VD),Nyon,Disability insurance (AI),Autre raison,French,Male,Disability Insurance (DI)
419416,Mathilde Cosandey-Meyer,+41 28 01 41 82,Lucerne (LU),Hochdorf,>70,Assurance vieillesse et survivants (AVS),Autre raison,19.10.2024 17:32:14,Lucerne (Lu),Hochdorf,Old age and survivors insurance (AVS),Autre raison,French,Female,Old-age and Survivors Insurance (OASI)
419417,Hélène Beuret,+41 00 32 26 43,Bâle-Campagne,Waldenburg,53–58,Assurance chômage (AC),Assistance pour postuler,01.01.2025 19:13:02,Basel-campaign,Waldenburg,Unemployment insurance (AC),Assistance pour postuler,French,Female,Unemployment Insurance (UI)
419418,Christophe Bourquin,+41 21 75 07 32,Fribourg (FR),Singine,16–22,Prévoyance professionnelle (LPP),Problème de formulaire,10.11.2024 03:38:35,Fruit (fr),Singine,Professional provident (LPP),Form problem,French,Female,Occupational Pension Plan (BVG)


In [31]:
df_sheet2 = df_sheet2.drop(columns=['Prestation','Prestation_translated'],errors='ignore')

In [32]:
df_sheet2

,Nom,Téléphone,Canton,District,Tranche d'âge,Problème,Horodatage,Canton_translated,District_translated,Problème_translated,Language,Gender,Scheme
0,Claude Béguelin,+41 61 17 49 01,Fribourg (FR),Sarine,41–46,Demande de pot-de-vin,05.11.2024 03:52:40,Fruit (fr),Sarine,Demande de pot-de-vin,French,Male,Health/Accident Insurance (LAMal/LAA)
1,Odette Chapuis,+41 10 79 23 88,Valais (VS),Viège,47–52,SMS d'approbation manquant,14.10.2024 17:31:53,Valais (vs),Threesome,SMS d'approbation manquant,French,Female,Disability Insurance (DI)
2,Robert Barillon,+41 06 90 09 33,Argovie (AG),Brugg,29–34,Autre raison,28.11.2024 10:14:14,Argovie (AG),Brugg,Autre raison,French,Male,Old-age and Survivors Insurance (OASI)
3,Philippe Martin,+41 09 53 06 62,Soleure (SO),Thal-Gäu,23–28,NaN,19.01.2025 06:01:03,SOOL (SO),Thal-Gäu,NaN,French,Male,Disability Insurance (DI)
4,Jonathan Mercier,+41 07 81 25 64,Appenzell,Oberegg,35–40,Autre raison,01.01.2025 07:27:26,Appenzell,Oberegg,Autre raison,French,Male,Unemployment Insurance (UI)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
419415,Jacques Niquille,+41 38 91 68 43,Vaud (VD),Nyon,16–22,Autre raison,19.10.2024 08:17:43,Vaud (VD),Nyon,Autre raison,French,Male,Disability Insurance (DI)
419416,Mathilde Cosandey-Meyer,+41 28 01 41 82,Lucerne (LU),Hochdorf,>70,Autre raison,19.10.2024 17:32:14,Lucerne (Lu),Hochdorf,Autre raison,French,Female,Old-age and Survivors Insurance (OASI)
419417,Hélène Beuret,+41 00 32 26 43,Bâle-Campagne,Waldenburg,53–58,Assistance pour postuler,01.01.2025 19:13:02,Basel-campaign,Waldenburg,Assistance pour postuler,French,Female,Unemployment Insurance (UI)
419418,Christophe Bourquin,+41 21 75 07 32,Fribourg (FR),Singine,16–22,Problème de formulaire,10.11.2024 03:38:35,Fruit (fr),Singine,Form problem,French,Female,Occupational Pension Plan (BVG)


district

In [33]:
df4 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french_District.xlsx')

In [34]:
df_sheet2 = pd.merge(df_sheet2, df4, on='District_translated', how='left')

In [35]:
df_sheet2 = df_sheet2.drop(columns=['District_x','District_translated'],errors='ignore')

ISSUES

In [36]:
df5 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french Issue.xlsx')

In [37]:
df5

,Problème_translated,Issue
0,Demande de pot-de-vin,Demand for bribe
1,SMS d'approbation manquant,Missing approval SMS
2,Autre raison,Other reason
3,NaN,NaN
4,Assistance pour postuler,Help applying
5,Form problem,Form issue
6,Paiement en attente,Pending payment
7,Site/Application problem,Website/Application trouble
8,Document/account problem,Document/account issue
9,Service problem,Benefit issue


In [38]:
# Perform the merge
df_sheet2 = pd.merge(df_sheet2, df5, on='Problème_translated', how='left')

In [39]:
df_sheet2 = df_sheet2.drop(columns=['Problème','Problème_translated'],errors='ignore')

Constituenc

In [40]:
df6 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\french_Canton.xlsx')

In [41]:
# Perform the merge
df_sheet2 = pd.merge(df_sheet2, df6, on='Canton_translated', how='left')


In [42]:
df_sheet2 = df_sheet2.drop(columns=['Canton_x','Canton_translated'],errors='ignore')

In [43]:
df_sheet2.head(5)

,Nom,Téléphone,Tranche d'âge,Horodatage,Language,Gender,Scheme,District_y,Issue,Canton_y
0,Claude Béguelin,+41 61 17 49 01,41–46,05.11.2024 03:52:40,French,Male,Health/Accident Insurance (LAMal/LAA),Sarine,Demand for bribe,Fribourg (FR)
1,Odette Chapuis,+41 10 79 23 88,47–52,14.10.2024 17:31:53,French,Female,Disability Insurance (DI),Visp,Missing approval SMS,Valais (VS)
2,Robert Barillon,+41 06 90 09 33,29–34,28.11.2024 10:14:14,French,Male,Old-age and Survivors Insurance (OASI),Brugg,Other reason,Aargau (AG)
3,Philippe Martin,+41 09 53 06 62,23–28,19.01.2025 06:01:03,French,Male,Disability Insurance (DI),Thal-Gäu,NaN,NaN
4,Jonathan Mercier,+41 07 81 25 64,35–40,01.01.2025 07:27:26,French,Male,Unemployment Insurance (UI),Oberegg,Other reason,Appenzell


renaming column

In [44]:
# Corrected column rename map
column_rename_map = {
    'Nom': 'Name',
    'Téléphone': 'Mobile Number',
    "Tranche d'âge": 'Age Group',
    'Horodatage': 'Timestamp',   # Renaming 'Update Time' to 'Date'
    'Language': 'Language',
    'Gender': 'Gender',
    'District_y': 'District',
    'Canton_y': 'Canton',

}

# Rename the columns in the DataFrame
df_sheet2.rename(columns=column_rename_map, inplace=True)

In [45]:
df_sheet2 = df_sheet2.fillna("")

In [46]:
df_sheet2.head()

,Name,Mobile Number,Age Group,Timestamp,Language,Gender,Scheme,District,Issue,Canton
0,Claude Béguelin,+41 61 17 49 01,41–46,05.11.2024 03:52:40,French,Male,Health/Accident Insurance (LAMal/LAA),Sarine,Demand for bribe,Fribourg (FR)
1,Odette Chapuis,+41 10 79 23 88,47–52,14.10.2024 17:31:53,French,Female,Disability Insurance (DI),Visp,Missing approval SMS,Valais (VS)
2,Robert Barillon,+41 06 90 09 33,29–34,28.11.2024 10:14:14,French,Male,Old-age and Survivors Insurance (OASI),Brugg,Other reason,Aargau (AG)
3,Philippe Martin,+41 09 53 06 62,23–28,19.01.2025 06:01:03,French,Male,Disability Insurance (DI),Thal-Gäu,,
4,Jonathan Mercier,+41 07 81 25 64,35–40,01.01.2025 07:27:26,French,Male,Unemployment Insurance (UI),Oberegg,Other reason,Appenzell


In [47]:
new_column_order = [
    "Name", "Mobile Number", "Age Group", "Language", "Gender",
    "Canton", "District", "Scheme", "Issue", "Timestamp"
]

# Strip whitespace (just in case)
df_sheet2.columns = df_sheet2.columns.str.strip()

# Filter to only include columns that exist
existing_columns = [col for col in new_column_order if col in df_sheet2.columns]

# Reorder DataFrame
df_sheet2 = df_sheet2[existing_columns]

In [48]:
df_sheet2 = pd.merge(df_sheet2, df25, on='Issue', how='left')

In [49]:
df_sheet2.head(5)

,Name,Mobile Number,Age Group,Language,Gender,Canton,District,Scheme,Issue,Timestamp,Call center
0,Claude Béguelin,+41 61 17 49 01,41–46,French,Male,Fribourg (FR),Sarine,Health/Accident Insurance (LAMal/LAA),Demand for bribe,05.11.2024 03:52:40,MLA/Party Office
1,Odette Chapuis,+41 10 79 23 88,47–52,French,Female,Valais (VS),Visp,Disability Insurance (DI),Missing approval SMS,14.10.2024 17:31:53,Division Office
2,Robert Barillon,+41 06 90 09 33,29–34,French,Male,Aargau (AG),Brugg,Old-age and Survivors Insurance (OASI),Other reason,28.11.2024 10:14:14,Our Call Center
3,Philippe Martin,+41 09 53 06 62,23–28,French,Male,,Thal-Gäu,Disability Insurance (DI),,19.01.2025 06:01:03,NaN
4,Jonathan Mercier,+41 07 81 25 64,35–40,French,Male,Appenzell,Oberegg,Unemployment Insurance (UI),Other reason,01.01.2025 07:27:26,Our Call Center


# German column standardization

In [50]:
df_sheet3 = df.parse('Deutsch 1')
df_sheet4 = df.parse('Deutsch 2')

In [51]:
df_sheet3.head()

,Name,Telefonnummer,Sprach,Kanton,Bezirk,Geschlecht,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Sprach_translated,Kanton_translated,Bezirk_translated,Geschlecht_translated,Leistung_translated,Unterstützung_translated
0,Marion Ritter,+41 77 07 44 41,Deutsch,Berne (BE),Seeland,Männlich,>70,NaN,NaN,13.01.2025 11:52:22,Deutsch,Berne (BE),Seeland,Masculine,NaN,NaN
1,Predrag Michel,+41 92 76 74 76,Deutsch,Vaud (VD),Lavaux-Oron,Weiblich,35–40,Invalidenversicherung (IV),Bestechung verlangt,03.01.2025 18:50:00,Deutsch,Vaud (VD),Lavaux-Oron,Weiblich,Invalid insurance (IV),Bestechung verlangt
2,Dolores Ott,+41 59 00 00 20,Deutsch,Zurich (ZH),Hinwil,Weiblich,41–46,Arbeitslosenversicherung (ALV),Problem mit Leistung,03.11.2024 04:37:12,Deutsch,Zurich (ZH),Hinwil,Weiblich,Unemployment insurance (ALV),Problem mit Leistung
3,Maxime Senn,+41 82 43 55 91,Deutsch,Uri (UR),Uri,Männlich,47–52,Arbeitslosenversicherung (ALV),Website-/App-Problem,10.10.2024 22:12:55,Deutsch,Uri (ur),Uri,Masculine,Unemployment insurance (ALV),Website-/App-Problem
4,Besim Eugster,+41 43 07 49 16,Deutsch,Berne (BE),Berne-Mittelland,Männlich,53–58,Alters- und Hinterlassenenversicherung (AHV),Zahlung ausstehend,16.12.2024 23:48:14,Deutsch,Berne (BE),Berne-Mittelland,Masculine,Age and left-wing insurance (AHV),Zahlung ausstehend


In [52]:
df_sheet4.head()

,Name,Telefonnummer,Sprach,Kanton,Bezirk,Geschlecht,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Sprach_translated,Kanton_translated,Bezirk_translated,Geschlecht_translated,Leistung_translated,Unterstützung_translated
0,Gil Ziegler-Grob,+41 30 75 21 93,Deutsch,Bâle-Ville (BS),Bâle-Ville,NaN,NaN,NaN,NaN,19.01.2025 08:31:52,Deutsch,Basel-city (BS),Basel-city,NaN,NaN,NaN
1,Prof. Paul Lehmann,+41 14 42 14 78,Deutsch,Valais (VS),Rarogne,Männlich,59–64,Kranken-/Unfallversicherung (KVG/UVG),Zahlung ausstehend,18.01.2025 19:26:04,Deutsch,Valais (vs),Rarogne,Masculine,Health/accident insurance (KVG/UVG),Zahlung ausstehend
2,Edmund Iten,+41 01 48 57 64,Deutsch,Vaud (VD),Lavaux-Oron,Männlich,53–58,Arbeitslosenversicherung (ALV),NaN,21.10.2024 00:00:10,Deutsch,Vaud (VD),Lavaux-Oron,Masculine,Unemployment insurance (ALV),NaN
3,Joel Scheidegger,+41 96 23 53 11,Deutsch,Bâle-Campagne,Waldenburg,Männlich,47–52,NaN,NaN,03.11.2024 13:38:20,Deutsch,Basel-campaign,Waldenburg,Masculine,NaN,NaN
4,Ismet Studer-Imhof,+41 83 71 13 28,Deutsch,Grisons (GR),Surselva,Weiblich,35–40,Invalidenversicherung (IV),Problem mit Leistung,12.10.2024 21:15:40,Deutsch,Grayons (GR),Surselva,Weiblich,Invalid insurance (IV),Problem mit Leistung


language

In [53]:
df7 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german_Language.xlsx')

In [54]:
df7 

,Sprach_translated,Language
0,Deutsch,Germany


In [55]:
df_sheet3 = pd.merge(df_sheet3, df7, on='Sprach_translated', how='left')

In [56]:
df_sheet4 = pd.merge(df_sheet4, df7, on='Sprach_translated', how='left')

In [57]:
df_sheet3 = df_sheet3.drop(columns=['Sprach','Sprach_translated'],errors='ignore')

In [58]:
df_sheet4 = df_sheet4.drop(columns=['Sprach','Sprach_translated'],errors='ignore')

In [59]:
df_sheet3.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Geschlecht,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Geschlecht_translated,Leistung_translated,Unterstützung_translated,Language
0,Marion Ritter,+41 77 07 44 41,Berne (BE),Seeland,Männlich,>70,NaN,NaN,13.01.2025 11:52:22,Berne (BE),Seeland,Masculine,NaN,NaN,Germany
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),Lavaux-Oron,Weiblich,35–40,Invalidenversicherung (IV),Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Lavaux-Oron,Weiblich,Invalid insurance (IV),Bestechung verlangt,Germany
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),Hinwil,Weiblich,41–46,Arbeitslosenversicherung (ALV),Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Hinwil,Weiblich,Unemployment insurance (ALV),Problem mit Leistung,Germany
3,Maxime Senn,+41 82 43 55 91,Uri (UR),Uri,Männlich,47–52,Arbeitslosenversicherung (ALV),Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Uri,Masculine,Unemployment insurance (ALV),Website-/App-Problem,Germany
4,Besim Eugster,+41 43 07 49 16,Berne (BE),Berne-Mittelland,Männlich,53–58,Alters- und Hinterlassenenversicherung (AHV),Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Berne-Mittelland,Masculine,Age and left-wing insurance (AHV),Zahlung ausstehend,Germany


In [60]:
df_sheet4.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Geschlecht,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Geschlecht_translated,Leistung_translated,Unterstützung_translated,Language
0,Gil Ziegler-Grob,+41 30 75 21 93,Bâle-Ville (BS),Bâle-Ville,NaN,NaN,NaN,NaN,19.01.2025 08:31:52,Basel-city (BS),Basel-city,NaN,NaN,NaN,Germany
1,Prof. Paul Lehmann,+41 14 42 14 78,Valais (VS),Rarogne,Männlich,59–64,Kranken-/Unfallversicherung (KVG/UVG),Zahlung ausstehend,18.01.2025 19:26:04,Valais (vs),Rarogne,Masculine,Health/accident insurance (KVG/UVG),Zahlung ausstehend,Germany
2,Edmund Iten,+41 01 48 57 64,Vaud (VD),Lavaux-Oron,Männlich,53–58,Arbeitslosenversicherung (ALV),NaN,21.10.2024 00:00:10,Vaud (VD),Lavaux-Oron,Masculine,Unemployment insurance (ALV),NaN,Germany
3,Joel Scheidegger,+41 96 23 53 11,Bâle-Campagne,Waldenburg,Männlich,47–52,NaN,NaN,03.11.2024 13:38:20,Basel-campaign,Waldenburg,Masculine,NaN,NaN,Germany
4,Ismet Studer-Imhof,+41 83 71 13 28,Grisons (GR),Surselva,Weiblich,35–40,Invalidenversicherung (IV),Problem mit Leistung,12.10.2024 21:15:40,Grayons (GR),Surselva,Weiblich,Invalid insurance (IV),Problem mit Leistung,Germany


Gender

In [61]:
df7 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german_gender.xlsx')

In [62]:
df7.head()

,Geschlecht_translated,Gender
0,Masculine,Male
1,Weiblich,Female
2,NaN,NaN
3,Andere,Other


In [63]:
# Perform the merge
df_sheet3 = pd.merge(df_sheet3, df7, on='Geschlecht_translated', how='left')

In [64]:
# Perform the merge
df_sheet4 = pd.merge(df_sheet4, df7, on='Geschlecht_translated', how='left')

In [65]:
df_sheet3.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Geschlecht,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Geschlecht_translated,Leistung_translated,Unterstützung_translated,Language,Gender
0,Marion Ritter,+41 77 07 44 41,Berne (BE),Seeland,Männlich,>70,NaN,NaN,13.01.2025 11:52:22,Berne (BE),Seeland,Masculine,NaN,NaN,Germany,Male
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),Lavaux-Oron,Weiblich,35–40,Invalidenversicherung (IV),Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Lavaux-Oron,Weiblich,Invalid insurance (IV),Bestechung verlangt,Germany,Female
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),Hinwil,Weiblich,41–46,Arbeitslosenversicherung (ALV),Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Hinwil,Weiblich,Unemployment insurance (ALV),Problem mit Leistung,Germany,Female
3,Maxime Senn,+41 82 43 55 91,Uri (UR),Uri,Männlich,47–52,Arbeitslosenversicherung (ALV),Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Uri,Masculine,Unemployment insurance (ALV),Website-/App-Problem,Germany,Male
4,Besim Eugster,+41 43 07 49 16,Berne (BE),Berne-Mittelland,Männlich,53–58,Alters- und Hinterlassenenversicherung (AHV),Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Berne-Mittelland,Masculine,Age and left-wing insurance (AHV),Zahlung ausstehend,Germany,Male


In [66]:
df_sheet3 = df_sheet3.drop(columns=['Geschlecht','Geschlecht_translated'],errors='ignore')

In [67]:
df_sheet4 = df_sheet4.drop(columns=['Geschlecht','Geschlecht_translated'],errors='ignore')

In [68]:
df_sheet3.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Leistung_translated,Unterstützung_translated,Language,Gender
0,Marion Ritter,+41 77 07 44 41,Berne (BE),Seeland,>70,NaN,NaN,13.01.2025 11:52:22,Berne (BE),Seeland,NaN,NaN,Germany,Male
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),Lavaux-Oron,35–40,Invalidenversicherung (IV),Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Lavaux-Oron,Invalid insurance (IV),Bestechung verlangt,Germany,Female
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),Hinwil,41–46,Arbeitslosenversicherung (ALV),Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Hinwil,Unemployment insurance (ALV),Problem mit Leistung,Germany,Female
3,Maxime Senn,+41 82 43 55 91,Uri (UR),Uri,47–52,Arbeitslosenversicherung (ALV),Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Uri,Unemployment insurance (ALV),Website-/App-Problem,Germany,Male
4,Besim Eugster,+41 43 07 49 16,Berne (BE),Berne-Mittelland,53–58,Alters- und Hinterlassenenversicherung (AHV),Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Berne-Mittelland,Age and left-wing insurance (AHV),Zahlung ausstehend,Germany,Male


scheme

In [69]:
df8 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german_scheme.xlsx')

In [70]:
df8

,Leistung_translated,Scheme
0,NaN,NaN
1,Invalid insurance (IV),Disability Insurance (DI)
2,Unemployment insurance (ALV),Unemployment Insurance (UI)
3,Age and left-wing insurance (AHV),Old-age and Survivors Insurance (OASI)
4,Family allowances (Fa),Family Allowances (FA)
5,Health/accident insurance (KVG/UVG),Health/Accident Insurance (LAMal/LAA)
6,Professional provision (BVG),Occupational Pension Plan (BVG)


In [71]:
# Perform the merge
df_sheet3 = pd.merge(df_sheet3, df8, on='Leistung_translated', how='left')

In [72]:
df_sheet3.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Altersgruppe,Leistung,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Leistung_translated,Unterstützung_translated,Language,Gender,Scheme
0,Marion Ritter,+41 77 07 44 41,Berne (BE),Seeland,>70,NaN,NaN,13.01.2025 11:52:22,Berne (BE),Seeland,NaN,NaN,Germany,Male,NaN
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),Lavaux-Oron,35–40,Invalidenversicherung (IV),Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Lavaux-Oron,Invalid insurance (IV),Bestechung verlangt,Germany,Female,Disability Insurance (DI)
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),Hinwil,41–46,Arbeitslosenversicherung (ALV),Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Hinwil,Unemployment insurance (ALV),Problem mit Leistung,Germany,Female,Unemployment Insurance (UI)
3,Maxime Senn,+41 82 43 55 91,Uri (UR),Uri,47–52,Arbeitslosenversicherung (ALV),Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Uri,Unemployment insurance (ALV),Website-/App-Problem,Germany,Male,Unemployment Insurance (UI)
4,Besim Eugster,+41 43 07 49 16,Berne (BE),Berne-Mittelland,53–58,Alters- und Hinterlassenenversicherung (AHV),Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Berne-Mittelland,Age and left-wing insurance (AHV),Zahlung ausstehend,Germany,Male,Old-age and Survivors Insurance (OASI)


In [73]:
# Perform the merge
df_sheet4 = pd.merge(df_sheet4, df8, on='Leistung_translated', how='left')

In [74]:
df_sheet3 = df_sheet3.drop(columns=['Leistung','Leistung_translated'],errors='ignore')

In [75]:
df_sheet4 = df_sheet4.drop(columns=['Leistung','Leistung_translated'],errors='ignore')

In [76]:
df_sheet3.head(5)

,Name,Telefonnummer,Kanton,Bezirk,Altersgruppe,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Bezirk_translated,Unterstützung_translated,Language,Gender,Scheme
0,Marion Ritter,+41 77 07 44 41,Berne (BE),Seeland,>70,NaN,13.01.2025 11:52:22,Berne (BE),Seeland,NaN,Germany,Male,NaN
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),Lavaux-Oron,35–40,Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Lavaux-Oron,Bestechung verlangt,Germany,Female,Disability Insurance (DI)
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),Hinwil,41–46,Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Hinwil,Problem mit Leistung,Germany,Female,Unemployment Insurance (UI)
3,Maxime Senn,+41 82 43 55 91,Uri (UR),Uri,47–52,Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Uri,Website-/App-Problem,Germany,Male,Unemployment Insurance (UI)
4,Besim Eugster,+41 43 07 49 16,Berne (BE),Berne-Mittelland,53–58,Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Berne-Mittelland,Zahlung ausstehend,Germany,Male,Old-age and Survivors Insurance (OASI)


district

In [77]:
df9 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german_District.xlsx')

In [78]:
df9

,Bezirk_translated,District
0,Affoltern,Affoltern
1,Aigle,Aigle
2,Albula,Albula
3,Andelfingen,Andelfingen
4,Appenzell,Appenzell
...,...,...
123,Wil,Wil
124,Willisau,Willisau
125,Winterthour,Winterthur
126,Zoug,Zug


In [79]:
df_sheet3 = pd.merge(df_sheet3, df9, on='Bezirk_translated', how='left')

In [80]:
df_sheet4 = pd.merge(df_sheet4, df9, on='Bezirk_translated', how='left')

In [81]:
df_sheet4 = df_sheet4.drop(columns=['Bezirk_translated','Bezirk'],errors='ignore')

In [82]:
df_sheet3 = df_sheet3.drop(columns=['Bezirk','Bezirk_translated'],errors='ignore')

In [83]:
df_sheet3.head()

,Name,Telefonnummer,Kanton,Altersgruppe,Unterstützung,Aktualisierungszeitpunkt,Kanton_translated,Unterstützung_translated,Language,Gender,Scheme,District
0,Marion Ritter,+41 77 07 44 41,Berne (BE),>70,NaN,13.01.2025 11:52:22,Berne (BE),NaN,Germany,Male,NaN,Seeland
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),35–40,Bestechung verlangt,03.01.2025 18:50:00,Vaud (VD),Bestechung verlangt,Germany,Female,Disability Insurance (DI),Lavaux-Oron
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),41–46,Problem mit Leistung,03.11.2024 04:37:12,Zurich (ZH),Problem mit Leistung,Germany,Female,Unemployment Insurance (UI),Hinwil
3,Maxime Senn,+41 82 43 55 91,Uri (UR),47–52,Website-/App-Problem,10.10.2024 22:12:55,Uri (ur),Website-/App-Problem,Germany,Male,Unemployment Insurance (UI),Uri
4,Besim Eugster,+41 43 07 49 16,Berne (BE),53–58,Zahlung ausstehend,16.12.2024 23:48:14,Berne (BE),Zahlung ausstehend,Germany,Male,Old-age and Survivors Insurance (OASI),Bern-Mittelland


ISSUES

In [84]:
df10 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german _issue.xlsx')

In [85]:
df10

,Unterstützung_translated,Issue
0,Bestechung verlangt,Demand for bribe
1,Problem mit Leistung,Benefit issue
2,Website-/App-Problem,Website/Application trouble
3,Zahlung ausstehend,Pending payment
4,Hilfe bei Antrag,Help applying
5,Behindertenhilfe,Disabled assistance
6,Formularproblem,Form issue
7,Anderer Grund,Other reason
8,SMS zur Genehmigung fehlt,Missing approval SMS
9,Document- / bank problem,Document/account issue


In [86]:
# Perform the merge
df_sheet3 = pd.merge(df_sheet3, df10, on='Unterstützung_translated', how='left')

In [87]:
# Perform the merge
df_sheet4 = pd.merge(df_sheet4, df10, on='Unterstützung_translated', how='left')

In [88]:
df_sheet3 = df_sheet3.drop(columns=['Unterstützung','Unterstützung_translated'],errors='ignore')

In [89]:
df_sheet4 = df_sheet4.drop(columns=['Unterstützung','Unterstützung_translated'],errors='ignore')

In [90]:
df_sheet3.head()

,Name,Telefonnummer,Kanton,Altersgruppe,Aktualisierungszeitpunkt,Kanton_translated,Language,Gender,Scheme,District,Issue
0,Marion Ritter,+41 77 07 44 41,Berne (BE),>70,13.01.2025 11:52:22,Berne (BE),Germany,Male,NaN,Seeland,NaN
1,Predrag Michel,+41 92 76 74 76,Vaud (VD),35–40,03.01.2025 18:50:00,Vaud (VD),Germany,Female,Disability Insurance (DI),Lavaux-Oron,Demand for bribe
2,Dolores Ott,+41 59 00 00 20,Zurich (ZH),41–46,03.11.2024 04:37:12,Zurich (ZH),Germany,Female,Unemployment Insurance (UI),Hinwil,Benefit issue
3,Maxime Senn,+41 82 43 55 91,Uri (UR),47–52,10.10.2024 22:12:55,Uri (ur),Germany,Male,Unemployment Insurance (UI),Uri,Website/Application trouble
4,Besim Eugster,+41 43 07 49 16,Berne (BE),53–58,16.12.2024 23:48:14,Berne (BE),Germany,Male,Old-age and Survivors Insurance (OASI),Bern-Mittelland,Pending payment


conton

In [91]:
df11 = pd.read_excel(r'C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\Columns File\german_Canton.xlsx')

In [92]:
df11

,Kanton_translated,Canton
0,Appenzell,Appenzell
1,Bâle-Campagne,Basel-Landschaft
2,Bâle-Ville (BS),Basel-Stadt (BS)
3,Berne (BE),Bern (BE)
4,Fribourg (FR),Fribourg (FR)
5,Genève (GE),Geneva (GE)
6,Glaris (GL),Glarus (GL)
7,Grisons (GR),Grisons (GR)
8,Jura (JU),Jura (JU)
9,Lucerne (LU),Lucerne (LU)


In [93]:
# Perform the merge
df_sheet3 = pd.merge(df_sheet3, df11, on='Kanton_translated', how='left')


In [94]:
# Perform the merge
df_sheet4 = pd.merge(df_sheet4, df11, on='Kanton_translated', how='left')


In [95]:
df_sheet3 = df_sheet3.drop(columns=['Kanton_translated','Kanton'],errors='ignore')

In [96]:
df_sheet4 = df_sheet4.drop(columns=['Kanton_translated','Kanton'],errors='ignore')

In [97]:
df_sheet3.head(5)

,Name,Telefonnummer,Altersgruppe,Aktualisierungszeitpunkt,Language,Gender,Scheme,District,Issue,Canton
0,Marion Ritter,+41 77 07 44 41,>70,13.01.2025 11:52:22,Germany,Male,NaN,Seeland,NaN,Bern (BE)
1,Predrag Michel,+41 92 76 74 76,35–40,03.01.2025 18:50:00,Germany,Female,Disability Insurance (DI),Lavaux-Oron,Demand for bribe,Vaud (VD)
2,Dolores Ott,+41 59 00 00 20,41–46,03.11.2024 04:37:12,Germany,Female,Unemployment Insurance (UI),Hinwil,Benefit issue,Zurich (ZH)
3,Maxime Senn,+41 82 43 55 91,47–52,10.10.2024 22:12:55,Germany,Male,Unemployment Insurance (UI),Uri,Website/Application trouble,NaN
4,Besim Eugster,+41 43 07 49 16,53–58,16.12.2024 23:48:14,Germany,Male,Old-age and Survivors Insurance (OASI),Bern-Mittelland,Pending payment,Bern (BE)


renaming column

In [98]:
# Corrected column rename map
column_rename_map = {
    'Name': 'Name',
    'Telefonnummer': 'Mobile Number',
    "Altersgruppe": 'Age Group',
    'Aktualisierungszeitpunkt': 'Timestamp',   # Renaming 'Update Time' to 'Date'
    'Language': 'Language',
    'Gender': 'Gender',
    'District_y': 'District',
    'Canton_y': 'Canton',

}

# Rename the columns in the DataFrame
df_sheet3.rename(columns=column_rename_map, inplace=True)
df_sheet4.rename(columns=column_rename_map, inplace=True)

In [99]:
df_sheet3 = df_sheet3.fillna("")
df_sheet4 = df_sheet4.fillna("")

In [100]:
df_sheet3.head(5)


,Name,Mobile Number,Age Group,Timestamp,Language,Gender,Scheme,District,Issue,Canton
0,Marion Ritter,+41 77 07 44 41,>70,13.01.2025 11:52:22,Germany,Male,,Seeland,,Bern (BE)
1,Predrag Michel,+41 92 76 74 76,35–40,03.01.2025 18:50:00,Germany,Female,Disability Insurance (DI),Lavaux-Oron,Demand for bribe,Vaud (VD)
2,Dolores Ott,+41 59 00 00 20,41–46,03.11.2024 04:37:12,Germany,Female,Unemployment Insurance (UI),Hinwil,Benefit issue,Zurich (ZH)
3,Maxime Senn,+41 82 43 55 91,47–52,10.10.2024 22:12:55,Germany,Male,Unemployment Insurance (UI),Uri,Website/Application trouble,
4,Besim Eugster,+41 43 07 49 16,53–58,16.12.2024 23:48:14,Germany,Male,Old-age and Survivors Insurance (OASI),Bern-Mittelland,Pending payment,Bern (BE)


In [101]:
df_sheet4.head()

,Name,Mobile Number,Age Group,Timestamp,Language,Gender,Scheme,District,Issue,Canton
0,Gil Ziegler-Grob,+41 30 75 21 93,,19.01.2025 08:31:52,Germany,,,Basel-Stadt,,
1,Prof. Paul Lehmann,+41 14 42 14 78,59–64,18.01.2025 19:26:04,Germany,Male,Health/Accident Insurance (LAMal/LAA),Rarogne,Pending payment,
2,Edmund Iten,+41 01 48 57 64,53–58,21.10.2024 00:00:10,Germany,Male,Unemployment Insurance (UI),Lavaux-Oron,,Vaud (VD)
3,Joel Scheidegger,+41 96 23 53 11,47–52,03.11.2024 13:38:20,Germany,Male,,Waldenburg,,
4,Ismet Studer-Imhof,+41 83 71 13 28,35–40,12.10.2024 21:15:40,Germany,Female,Disability Insurance (DI),Surselva,Benefit issue,


In [102]:
# Corrected column rename map
column_rename_map = {
    'Language_x': 'Language',
}

# Rename the columns in the DataFrame

df_sheet4.rename(columns=column_rename_map, inplace=True)
df_sheet4 = df_sheet4.drop(columns=['Language_y'],errors='ignore')

In [103]:
new_column_order = [
    "Name", "Mobile Number", "Age Group", "Language", "Gender",
    "Canton", "District", "Scheme", "Issue","Timestamp"
]

# Reorder the columns
df_sheet3 = df_sheet3[new_column_order]
df_sheet4 = df_sheet4[new_column_order]

In [104]:
df_sheet3 = pd.merge(df_sheet3, df25, on='Issue', how='left')

In [105]:
df_sheet4 = pd.merge(df_sheet4, df25, on='Issue', how='left')

In [106]:
df_sheet3.head(5)

,Name,Mobile Number,Age Group,Language,Gender,Canton,District,Scheme,Issue,Timestamp,Call center
0,Marion Ritter,+41 77 07 44 41,>70,Germany,Male,Bern (BE),Seeland,,,13.01.2025 11:52:22,NaN
1,Predrag Michel,+41 92 76 74 76,35–40,Germany,Female,Vaud (VD),Lavaux-Oron,Disability Insurance (DI),Demand for bribe,03.01.2025 18:50:00,MLA/Party Office
2,Dolores Ott,+41 59 00 00 20,41–46,Germany,Female,Zurich (ZH),Hinwil,Unemployment Insurance (UI),Benefit issue,03.11.2024 04:37:12,Our Call Center
3,Maxime Senn,+41 82 43 55 91,47–52,Germany,Male,,Uri,Unemployment Insurance (UI),Website/Application trouble,10.10.2024 22:12:55,MLA/Party Office
4,Besim Eugster,+41 43 07 49 16,53–58,Germany,Male,Bern (BE),Bern-Mittelland,Old-age and Survivors Insurance (OASI),Pending payment,16.12.2024 23:48:14,Our Call Center


In [107]:
df_sheet4.head(5)

,Name,Mobile Number,Age Group,Language,Gender,Canton,District,Scheme,Issue,Timestamp,Call center
0,Gil Ziegler-Grob,+41 30 75 21 93,,Germany,,,Basel-Stadt,,,19.01.2025 08:31:52,NaN
1,Prof. Paul Lehmann,+41 14 42 14 78,59–64,Germany,Male,,Rarogne,Health/Accident Insurance (LAMal/LAA),Pending payment,18.01.2025 19:26:04,Our Call Center
2,Edmund Iten,+41 01 48 57 64,53–58,Germany,Male,Vaud (VD),Lavaux-Oron,Unemployment Insurance (UI),,21.10.2024 00:00:10,NaN
3,Joel Scheidegger,+41 96 23 53 11,47–52,Germany,Male,,Waldenburg,,,03.11.2024 13:38:20,NaN
4,Ismet Studer-Imhof,+41 83 71 13 28,35–40,Germany,Female,,Surselva,Disability Insurance (DI),Benefit issue,12.10.2024 21:15:40,Our Call Center


## Export to excel

In [108]:
# Replace 'output_file.xlsx' with your desired filename
output_file = r"C:\Users\hp\Desktop\Github\Power-BI\1 ChatBot Journey Insights and Performance Dashboard\2 DAILY DATA CLEAN PROCESS CHAT BOT DATA\output_file_final.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_sheet1.to_excel(writer, sheet_name='Sheet1', index=False)
    df_sheet2.to_excel(writer, sheet_name='Sheet2', index=False)
    df_sheet3.to_excel(writer, sheet_name='Sheet3', index=False)
    df_sheet4.to_excel(writer, sheet_name='Sheet4', index=False)